In [1]:
import requests#hyper text tranfer protocol
import pandas as pd
import psycopg2

data extract

first data=competition,category

In [ ]:

# API URL
url = "https://api.sportradar.com/tennis/trial/v3/en/competitions.json?api_key=z2N24Ttb6eFyBTodloCy33iXUDGK07nZwWVMnq6t"

# API request
def fetch_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Error-statuscode: {response.status_code}")
try:
    data = fetch_data(url)#json-dic

    competitions = data.get("competitions", [])

    # separate lists  
    competition_details = []#competition details
    category_details = []#category details

    for competition in competitions:
        comp_detail = {
            "competition_id": competition.get("id"),
            "competition_name": competition.get("name"),
            "parent_id":competition.get("parent_id"),
            "type": competition.get("type"),
            "gender": competition.get("gender"),
            "category_id" : competition.get("category",{}).get("id"),
        }
        competition_details.append(comp_detail)

        
        category = competition.get("category", {})
        category_detail = {
            "category_id": category.get("id"),
            "category_name": category.get("name"),
        }
        category_details.append(category_detail)
    print("data inserted succesfully")
except Exception as e:
    print(e)

second data=competitior_ranking_table,competitors_table

In [ ]:

url = "https://api.sportradar.com/tennis/trial/v3/en/double_competitors_rankings.json?api_key=z2N24Ttb6eFyBTodloCy33iXUDGK07nZwWVMnq6t"

def fetch_data(url):
    response=requests.get(url)
    if response.status_code==200:
        return response.json()
    else:
        raise Exception (f"Error:{response.status_code}")
try:
    data=fetch_data(url)

    Competitors_Table=[]
    Competitor_Rankings_Table=[]

    for ranking in data.get("rankings", []):
        for competitor_ranking in ranking.get("competitor_rankings", []):
            Rank_data = {
                "rank": competitor_ranking.get("rank"),
                "movement": competitor_ranking.get("movement"),
                "points": competitor_ranking.get("points"),
                "competitions_played": competitor_ranking.get("competitions_played"),
                "competitor_id": competitor_ranking["competitor"].get("id"),
            }
            Competitor_Rankings_Table.append(Rank_data)
                
            compi= competitor_ranking.get("competitor",{})
            compx_info={
                "competitor_id":compi.get("id"),
                "name":compi.get("name"),
                "country":compi.get("country"),
                "country_code":compi.get("country_code"),
                "abbreviation":compi.get("abbreviation")
            }
            Competitors_Table.append(compx_info)
    print("Data is Success")
except Exception as e:
     print(e)

third data=complex_details_,venue_details

In [ ]:

url = "https://api.sportradar.com/tennis/trial/v3/en/complexes.json?api_key=z2N24Ttb6eFyBTodloCy33iXUDGK07nZwWVMnq6t"

# API request
def fetch_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Error-statuscode: {response.status_code}")
try:
    data=fetch_data(url)#json-dic

    complex_details = []#complex_details details
    venue_details = []#venue_details details

    complexes = data.get("complexes", [])
    for complex_item in complexes:
        comp_details={
            "complex_id":complex_item.get("id"),
            "complex_name":complex_item.get("name")
        }
        complex_details.append(comp_details)

        venues = complex_item.get("venues", [])
        for venue in venues:
                venue_detail = {
                    "venue_id": venue.get("id"),
                    "venue_name": venue.get("name"),
                    "city_name": venue.get("city_name"),
                    "country_name": venue.get("country_name"),
                    "country_code": venue.get("country_code"),
                    "timezone": venue.get("timezone"),
                    "complex_id": complex_item.get("id")  
                }
                venue_details.append(venue_detail)
    print("data inserted succesfully")
except Exception as e:
    print(e)

convert to CSV using pandas 

first data=competition,category

In [5]:
pd.DataFrame(competition_details).to_csv('competition_details.csv', index=False)
pd.DataFrame(category_details).to_csv('category_details.csv',index=False)

third data=complex_details_,venue_details

In [6]:
pd.DataFrame(complex_details).to_csv("complex_details.csv",index=False)
pd.DataFrame(venue_details).to_csv("venue_details.csv",index=False)

second data=competitior_ranking_table,competitors_table

In [7]:
pd.DataFrame(Competitor_Rankings_Table).to_csv('Competitor_Rankings_Table.csv',index=False)
pd.DataFrame(Competitors_Table).to_csv('Competitors_Table.csv',index=False)

In [ ]:
connection = psycopg2.connect(
    host="localhost",
    user="postgres",
    password="9346"
)

connection.autocommit=True

cursor = connection.cursor()
data_base=("create database BigData")
cursor.execute(data_base)
print("Data base created")
cursor.close()
connection.close()



inserting table into sql_1

In [ ]:
connection=psycopg2.connect(
    host="localhost",
    database="bigdata",
    user="postgres",
    password="9346"
)

connection.autocommit=True
cursor=connection.cursor()

cursor.execute("""
    CREATE TABLE categories (
        category_id VARCHAR(50) PRIMARY KEY,
        category_name VARCHAR(100));""")
print("table_1 categories created columns added")

cursor.execute("""
    create table competitions(
    competition_id varchar(50) primary key,
    competition_name varchar(100) ,
    parent_id varchar(50) ,
    type varchar(20),
    gender varchar(10) ,
    category_id varchar(50),
    foreign key (category_id) references categories(category_id));""")
print("table_2 competition created columns added")

inserting table into sql_2

In [ ]:

cursor.execute("""
CREATE TABLE complexes (
    complex_id VARCHAR(50)  PRIMARY KEY,
    complex_name VARCHAR(100));""")
print("table_3 complexes created columns added")

cursor.execute("""
CREATE TABLE venue (
    venue_id varchar(50) PRIMARY KEY, 
    venue_name varchar(100),
    city_name varchar(100) ,
    country_name varchar(100) ,
    country_code varchar(100),
    timezone varchar(100),
    complex_id varchar(50),
    FOREIGN KEY (complex_id) REFERENCES complexes(complex_id));""")
print("Table_4 venues created columns added")

inserting table into sql_3

In [ ]:


cursor.execute("""
CREATE TABLE Competitors (
    competitor_id VARCHAR(50) PRIMARY KEY,
    name VARCHAR(100),
    country VARCHAR(100),
    country_code CHAR(3),
    abbreviation VARCHAR(10));""")
print("table_5 competitors created column added")

cursor.execute("""
CREATE TABLE Competitor_Rankings (
    rank_id SERIAL PRIMARY KEY, 
    rank int,
    movement INT ,
    points INT ,
    competitions_played INT ,
    competitor_id VARCHAR(50),
    FOREIGN KEY (competitor_id) REFERENCES Competitors(competitor_id));""")
print("table_6 competitor_ranking created column added")


In [18]:
df = pd.read_csv('category_details.csv')
df1=pd.read_csv('competition_details.csv')
df2=pd.read_csv("complex_details.csv")
df3=pd.read_csv("venue_details.csv")
df4=pd.read_csv('Competitors_Table.csv')
df5=pd.read_csv('Competitor_Rankings_Table.csv')
df5['movement'] = df5['movement'].fillna(0)

In [ ]:
insert = "insert into categories (category_id, category_name) VALUES (%s, %s) ON CONFLICT (category_id) DO NOTHING"

for index, row in df.iterrows():
    cursor.execute(insert, (row['category_id'], row['category_name']))

insert_query="""insert into competitions(competition_id, competition_name, parent_id, type, gender, category_id)
VALUES (%s, %s, %s, %s, %s, %s)
ON CONFLICT (competition_id) DO UPDATE
SET
    competition_name = EXCLUDED.competition_name,
    parent_id = EXCLUDED.parent_id,
    type = EXCLUDED.type,
    gender = EXCLUDED.gender,
    category_id = EXCLUDED.category_id;"""

for index, row in df1.iterrows():
    cursor.execute(insert_query, (
        row['competition_id'], 
        row['competition_name'], 
        row['parent_id'], 
        row['type'], 
        row['gender'], 
        row['category_id']
    ))
print("success data")

In [ ]:
insert = "INSERT INTO complexes (complex_id, complex_name) VALUES (%s, %s)"

for index, row in df2.iterrows():
    cursor.execute(insert, (row['complex_id'], row['complex_name']))

insert_venue = """
INSERT INTO venue (venue_id, venue_name, city_name, country_name, country_code, timezone, complex_id) 
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

for index, row in df3.iterrows():
    cursor.execute(insert_venue, (
        row['venue_id'], 
        row['venue_name'], 
        row['city_name'], 
        row['country_name'], 
        row['country_code'], 
        row['timezone'], 
        row['complex_id']  
    ))

print("success data")

In [ ]:
insert = """ 
insert into competitors (competitor_id, name, country, country_code, abbreviation)
values (%s, %s, %s, %s, %s)
on conflict (competitor_id) do nothing
"""

for index, row in df4.iterrows():
    cursor.execute(insert, (
        row['competitor_id'], 
        row['name'], 
        row['country'], 
        row['country_code'], 
        row['abbreviation']
    ))

insert_query="""
insert into Competitor_Rankings(rank , movement , points , competitions_played , competitor_id) 
values (%s,%s,%s,%s,%s)
"""
for index,row in df5.iterrows():
    cursor.execute(insert_query,(
        row['rank'],
        row['movement'],
        row['points'],
        row['competitions_played'],
        row['competitor_id']
    ))

[query-1]Categories Table--Competitions Table=Data Analysis:
-Execute the Following SQL queries;
1.List all competitions along with their category name
2.Count the number of competitions in each category
3.Find all competitions of type 'doubles'
4.Get competitions that belong to a specific category (e.g., ITF Men)
5.Identify parent competitions and their sub-competitions
6.Analyze the distribution of competition types by category
7.List all competitions with no parent (top-level competitions)


1.List all competitions along with their category name

In [ ]:
q1 = """
SELECT 
    c.competition_id,
    c.competition_name,
    c.type,
    c.gender,
    c.category_id,
    cat.category_name
FROM 
    competitions c JOIN categories cat 
ON 
c.category_id = cat.category_id;
"""
cursor.execute(q1)
out_1 = cursor.fetchall()
print("competitons along with their category_name:--\n")
for fetch in out_1:
    print(fetch)


2.Count the number of competitions in each category

In [ ]:
q2 = """
SELECT 
    cat.category_name, 
    COUNT(c.competition_id) 
FROM 
    competitions c JOIN  categories cat 
ON
 c.category_id = cat.category_id
GROUP BY 
    cat.category_name;
"""
cursor.execute(q2)
out_2 = cursor.fetchall()
print("\n2. Number of competitions in each category:")
for fetch in out_2:
    print(fetch)

3.Find all competitions of type 'doubles'

In [ ]:
query3 = """
SELECT 
    competition_id,
    competition_name,
    type
FROM 
    competitions
WHERE 
    type = 'doubles';
"""
cursor.execute(query3)
result3 = cursor.fetchall()
print("\n3. All competitions of type 'doubles':")
for row in result3:
    print(row)

4.Get competitions that belong to a specific category (e.g., ITF Men)


In [ ]:
category_id = 'sr:category:3'  
query4 = f"""
SELECT 
    competition_id,
    competition_name,
    category_id
FROM 
    competitions
WHERE 
    category_id = '{category_id}';
"""
cursor.execute(query4)
result4 = cursor.fetchall()
print(f"\n4. Competitions of category {category_id}:")
for row in result4:
    print(row)

Query 5: Identify parent competitions and their sub-competition

In [ ]:
query5 = """
SELECT 
    parent.competition_id AS parent_id,
    parent.competition_name AS parent_name,
    sub.competition_id AS sub_competition_id,
    sub.competition_name AS sub_competition_name
FROM 
    competitions parent
JOIN 
    competitions sub ON parent.competition_id = sub.parent_id
WHERE 
    parent.parent_id IS NULL;
"""
cursor.execute(query5)
result5 = cursor.fetchall()
print("\n5. Parent competitions and their sub-competitions:")
for row in result5:
    print(row)

 Query 6: Analyze the distribution of competition types by category

In [ ]:
query6 = """
SELECT 
    cat.category_name,
    c.type,
    COUNT(c.competition_id) AS competition_count
FROM 
    competitions c
JOIN 
    categories cat ON c.category_id = cat.category_id
GROUP BY 
    cat.category_name, c.type
ORDER BY 
    cat.category_name, c.type;
"""
cursor.execute(query6)
result6 = cursor.fetchall()
print("\n6. Distribution of competition types by category:")
for row in result6:
    print(row)


Query 7: List all competitions with no parent (top-level competitions)

In [ ]:
query7 = """
SELECT 
    competition_id,
    competition_name
FROM 
    competitions
WHERE 
    parent_id is null;
"""
cursor.execute(query7)
result7 = cursor.fetchall()
print("\nall competitions with no parent (top-level competitions)")
for row in result7:
    print(row)

[query-2]Execute the following SQL queries:
1.List all venues along with their associated complex name
2.Count the number of venues in each complex
3.Get details of venues in a specific country (e.g., Chile)
4.Identify all venues and their timezones
5.Find complexes that have more than one venue
6.List venues grouped by country
7.Find all venues for a specific complex (e.g., Nacional)


In [ ]:
query1 = """
SELECT 
    v.venue_name, 
    c.complex_name
FROM 
    venue v
LEFT JOIN 
    complexes c 
ON 
    v.complex_id = c.complex_id;
"""
cursor.execute(query1)
result1 = cursor.fetchall()
print("\nVenues along with their associated complex name:")
for row in result1:
    print(row)

In [ ]:
query2 = """
SELECT 
    c.complex_name, 
    COUNT(v.venue_id) AS venue_count
FROM 
    complexes c
LEFT JOIN 
    venue v 
ON 
    c.complex_id = v.complex_id
GROUP BY 
    c.complex_name;
"""
cursor.execute(query2)
result2 = cursor.fetchall()
print("\nNumber of venues in each complex:")
for row in result2:
    print(row)

In [ ]:
query3 = """
SELECT 
    venue_name, 
    city_name, 
    country_name, 
    timezone
FROM 
    venue
WHERE 
    country_name = 'Chile';
"""
cursor.execute(query3)
result3 = cursor.fetchall()
print("\nVenues in Chile:")
for row in result3:
    print(row)

In [ ]:
query4 = """
SELECT 
    venue_name, 
    timezone
FROM 
    venue;
"""
cursor.execute(query4)
result4 = cursor.fetchall()
print("\nVenues and their timezones:")
for row in result4:
    print(row)

In [ ]:
query5 = """
SELECT 
    c.complex_name, 
    COUNT(v.venue_id) AS venue_count
FROM 
    complexes c
JOIN 
    venue v 
ON 
    c.complex_id = v.complex_id
GROUP BY 
    c.complex_name
HAVING 
    COUNT(v.venue_id) > 1;
"""
cursor.execute(query5)
result5 = cursor.fetchall()
print("\nComplexes with more than one venue:")
for row in result5:
    print(row)

In [ ]:
query6 = """
SELECT 
    country_name, 
    ARRAY_AGG(venue_name) AS venues
FROM 
    venue
GROUP BY 
    country_name;
"""
cursor.execute(query6)
result6 = cursor.fetchall()
print("\nVenues grouped by country:")
for row in result6:
    print(row)

7. Find all venues for a specific complex (e.g., Nacional)


In [ ]:
complex_name='Palais des sports de Gerland'
query7 =f"""
SELECT 
    venue_name
FROM 
    venue
WHERE 
    complex_id = (SELECT complex_id FROM complexes WHERE complex_name = '{complex_name}');
"""
cursor.execute(query7)
result7 = cursor.fetchall()
print(f"\nVenues for the Nacional complex:{complex_name}")
for row in result7:
    print(row)



In [ ]:
query1 = """
SELECT 
    cr.competitor_id, 
    cr.rank, 
    cr.points 
FROM 
    Competitor_Rankings cr;
"""
cursor.execute(query1)
result1 = cursor.fetchall()
print("\nAll competitors with their rank and points:")
for row in result1:
    print(row)

In [ ]:
query2 = """
SELECT 
    cr.competitor_id, 
    cr.rank, 
    cr.points 
FROM 
    Competitor_Rankings cr 
WHERE 
    cr.rank <= 5;
"""
cursor.execute(query2)
result2 = cursor.fetchall()
print("\nTop 5 ranked competitors:")
for row in result2:
    print(row)

In [ ]:
query3 = """
SELECT 
    cr.competitor_id, 
    cr.rank, 
    cr.points 
FROM 
    Competitor_Rankings cr 
WHERE 
    cr.movement = 0;
"""
cursor.execute(query3)
result3 = cursor.fetchall()
print("\nCompetitors with stable rank (no movement):")
for row in result3:
    print(row)


In [ ]:
query4 = """
SELECT 
    SUM(cr.points) 
FROM 
    Competitor_Rankings cr 
JOIN 
    Competitors c ON cr.competitor_id = c.competitor_id
WHERE 
    c.country = 'Croatia';
"""
cursor.execute(query4)
result4 = cursor.fetchall()
print("\nTotal points of competitors from Croatia:")
for row in result4:
    print(row)

In [ ]:
query5 = """
SELECT 
    c.country, 
    COUNT(cr.competitor_id) AS competitor_count
FROM 
    Competitor_Rankings cr
JOIN 
    Competitors c ON cr.competitor_id = c.competitor_id
GROUP BY 
    c.country;
"""
cursor.execute(query5)
result5 = cursor.fetchall()
print("\nNumber of competitors per country:")
for row in result5:
    print(row)

In [ ]:
query6 = """
SELECT 
    cr.competitor_id, 
    cr.rank, 
    cr.points 
FROM 
    Competitor_Rankings cr
ORDER BY 
    cr.points DESC 
LIMIT 1;
"""
cursor.execute(query6)
result6 = cursor.fetchall()
print("\nCompetitors with the highest points:")
for row in result6:
    print(row)
